In [137]:
import numpy as np
import pandas as pd
from krippendorff import krippendorff
#krippendorff.alpha(reliability_data=...)

In [11]:
all_data = pd.read_csv('data/si630w22-hw3-data.csv')
all_data.sample(5)

,question_id,question_text,reply_id,reply_text,rlen
3715,t3_nkj84m,People who were unsure whether or not to pursu...,gzd6psp,I met someone in a psychiatric clinic and I wa...,1132
2316,t3_noygid,What is your opinion on cryptocurrency and whe...,h02an9v,I think its definitely going to *change* how w...,751
282,t3_nhanfm,What’s something on the internet that scared y...,gyvt189,"the YouTube video ""the koolaid killer.' this w...",84
1499,t3_niddh5,What do you guys do when you guys are at your ...,gz1a4y8,If she doesn’t have a video game system worth ...,82
2976,t3_n4oh4s,"Non-Americans, what American fried food isn’t ...",gwwnzsf,:) I hung out with this German student who tol...,253


In [13]:
test_df = pd.read_csv('data/si630w22-hw3-train.csv')
test_df.group.unique()

array(['group_09', 'group_03', 'group_10', 'group_13', 'group_22',
       'group_01', 'group_02', 'group_20', 'group_04', 'group_12',
       'group_15', 'group_24', 'group_17', 'group_25', 'group_19',
       'group_08', 'group_07', 'group_18', 'group_21', 'group_16',
       'group_23', 'group_05', 'group_14', 'group_11'], dtype=object)

In [14]:
group23 = test_df[test_df.group == 'group_23'].copy()
print(group23.shape)
group23.sample(5, random_state=1)

(608, 4)


,id,annotator_id,rating,group
15238,t3_n7bf3w,user_50,4.0,group_23
15410,t3_nko8fo,user_50,4.0,group_23
15320,t3_nd8ssn,user_50,5.0,group_23
15638,t3_nei3i3,user_51,5.0,group_23
15360,t3_ngoh9z,user_50,4.0,group_23


In [33]:
group23_ids = group23.id.unique()
print(len(group23_ids))

304


In [17]:
merged_group23 = group23.merge(right=all_data, how='inner', left_on='id', right_on='question_id')
print(merged_group23.shape)
print(merged_group23.group.unique())
merged_group23.sample(5, random_state=1)

(608, 9)
['group_23']


,id,annotator_id,rating,group,question_id,question_text,reply_id,reply_text,rlen
65,t3_n4scb5,user_51,4.0,group_23,t3_n4scb5,Do you talk to yourself? To what extent?,gwxbok4,It's not weird at all. I do it mostly when I'm...,161
237,t3_nb2kny,user_51,5.0,group_23,t3_nb2kny,"eli5 : Why do we have ""glitch-like"" moment ? L...",gxx33fx,There's a lot of complex thought that comes fr...,568
147,t3_n85dk8,user_51,4.0,group_23,t3_n85dk8,What’s something you for sure know but always ...,gxgkk2n,"I'm good enough, I'm smart enough, and doggone...",66
465,t3_nkblfm,user_51,5.0,group_23,t3_nkblfm,Can you ask for proof of any truths forever? E...,gzcuffi,It's a perfectly good question! Some epistemol...,841
187,t3_n9msxp,user_51,5.0,group_23,t3_n9msxp,"People who got expelled from school, what did ...",gxoovro,"Dress code violation was the final straw, but ...",369


In [20]:
merged_group23.groupby('annotator_id').size()

annotator_id
user_50    304
user_51    304
dtype: int64

In [22]:
merged_group23.id.duplicated().count()

608

In [34]:
new_index_test = test_df.copy().set_index('id')
new_index_test.loc[list(group23_ids)].groupby('group').size()

group
group_01     26
group_02     36
group_03     22
group_04     18
group_05     32
group_07     42
group_08     26
group_09     36
group_10     36
group_11     39
group_12     36
group_13     24
group_14     24
group_15     30
group_16     22
group_17     48
group_18     42
group_19     26
group_20     36
group_21     24
group_22     36
group_23    608
group_24     36
group_25     24
dtype: int64

In [36]:
group23_overlap = new_index_test.loc[list(group23_ids)].copy()
group23_overlap_wqs = group23_overlap.merge(right=all_data, how='inner', left_on='id', right_on='question_id')

In [38]:
group23_overlap_wqs.head()

,annotator_id,rating,group,question_id,question_text,reply_id,reply_text,rlen
0,user_15,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
1,user_16,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
2,user_17,3.0,group_02,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
3,user_50,3.0,group_23,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42
4,user_51,3.0,group_23,t3_n2c9u0,What is One Item you recommend everyone to have?,gwii5or,Rule 9: Never go anywhere without a knife.,42


## Inter Annotator Agreement

In [111]:
only23 = group23_overlap_wqs[(group23_overlap_wqs.annotator_id == 'user_50') | (group23_overlap_wqs.annotator_id == 'user_51')].copy()
ratings23_df = only23.pivot_table(index='question_id', columns='annotator_id', values='rating')
print(ratings23_df.corr())

annotator_id   user_50   user_51
annotator_id                    
user_50       1.000000  0.957879
user_51       0.957879  1.000000


In [143]:
rd = np.array(only23.pivot_table(index='question_id', columns='annotator_id', values='rating').T)
print(rd.shape)
vd = np.array([1,2,3,4,5])
print(vd.shape)
vc = krippendorff._reliability_data_to_value_counts(rd, vd)
vc.shape

(2, 304)
(5,)


(304, 5)

In [144]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=vc,level_of_measurement="nominal"))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=vc))

Krippendorff's alpha for nominal metric:  0.9198968908908557
Krippendorff's alpha for interval metric:  0.9576501779111142


## Class-Wide Annotator Agreement

In [145]:
rd_all = np.array(group23_overlap_wqs.pivot_table(index='question_id', columns='annotator_id', values='rating').T)
print(rd_all.shape)
vd_all = np.array([1,2,3,4,5])
print(vd_all.shape)
vc_all = krippendorff._reliability_data_to_value_counts(rd_all, vd_all)
print(vc_all.shape)

(59, 304)
(5,)
(304, 5)


In [146]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=vc_all,level_of_measurement="nominal"))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=vc_all))

Krippendorff's alpha for nominal metric:  0.2953877580177522
Krippendorff's alpha for interval metric:  0.49413246108797215


**No need to do pearson correlation r on classwide data**

In [83]:
avg_ratings_bygroup = group23_overlap_wqs.pivot_table(index='question_id', columns='group', values='rating', aggfunc=np.mean)
groups_corr = pd.DataFrame(avg_ratings_df.corr().group_23).reset_index()
#groups_corr.to_csv('pearson_correlation_all_annotators.csv', index=False)
groups_corr

,group,group_23
0,group_01,0.469881
1,group_02,0.253562
2,group_03,0.041138
3,group_04,-0.473686
4,group_05,0.844626
5,group_07,0.564816
6,group_08,0.201569
7,group_09,0.618990
8,group_10,0.335148
9,group_11,0.590256


In [92]:
avg_ratings = group23_overlap_wqs.pivot_table(index='question_id', columns='group', values='rating', aggfunc=np.mean)
avg_ratings

group,group_01,group_02,group_03,group_04,group_05,group_07,group_08,group_09,group_10,group_11,...,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25
question_id,,,,,,,,,,,,,,,,,,,,,
t3_n2c9u0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
t3_n2hebp,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN
t3_n2kuid,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
t3_n2nc15,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN
t3_n2qa4v,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t3_npefe0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
t3_npezra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN
t3_npfe4x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,NaN,4.0,NaN,NaN


In [93]:
avg_ratings.corr()

group,group_01,group_02,group_03,group_04,group_05,group_07,group_08,group_09,group_10,group_11,...,group_16,group_17,group_18,group_19,group_20,group_21,group_22,group_23,group_24,group_25
group,,,,,,,,,,,,,,,,,,,,,
group_01,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.469881,NaN,NaN
group_02,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.253562,NaN,NaN
group_03,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041138,NaN,NaN
group_04,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.473686,NaN,NaN
group_05,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.844626,NaN,NaN
group_07,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.564816,NaN,NaN
group_08,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201569,NaN,NaN
group_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.618990,NaN,NaN
group_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335148,NaN,NaN
